This notebook walks through the steps that combine the following data sources:
- original SBA data with Yelp and Google review, as well as geo-coding
- 2015 number of businesses by employee size, NAICS code, and ZIP code 
- 2014 IRS tax return data by zip code

Although I explain where each file comes from, this script assumes that all three of the following files are present in the working directory (in a folder titled 'Data'):
- https://www.dropbox.com/s/a9h343yehh261ha/Clean_Yelp.csv?dl=1
- https://www.dropbox.com/s/em2s9fgkl1qajdu/zip_by_naics.csv?dl=1
- https://www.dropbox.com/s/po0uv43x6x1omzp/14_irs_zip.csv?dl=1

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import StringIO
import requests
%matplotlib inline

### Original SBA data, with Yelp and Google reviews, and Lat-Long  

Google reviews and goecoordinates came from Noah's dataset on Carto. I pulled in Yelp review data myself (includes average rating, number of review, and Yelp url). Yelp data obtained through Yelp Fusion API using addess and business name search and retrieving the top result (see _yelp.py_ file)

In [2]:
clean = pd.read_csv(os.path.join('Data', 'Clean_Yelp.csv'))

In [3]:
clean.shape

(43817, 63)

Get total number of SBA loans: aggregate, 540, and 7a.  
_Note_: we will not be using Yelp data for the aggregate analysis of 'overserved' and 'underserved.'

In [56]:
clean_tots = clean.groupby('borrzip').size()

In [57]:
clean_tots_540 = clean[clean['program'] == '504'].groupby('borrzip').size()

In [58]:
clean_tots_7a = clean[clean['program'] == '7(a)'].groupby('borrzip').size()

### 2015 Zip Data for number of businesses by employee size and NAICS code  

Original data obtained here: https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?src=bkmk  
Survey methodology: https://www.census.gov/programs-surveys/cbp/technical-documentation/methodology.html  
Downloaded file from my dropbox: https://www.dropbox.com/s/27xjbyp4bcp51pl/CB1500CZ21.dat?dl=0

#### NOTE: the following 6 steps pull the data from the original large dataset, which I've excluded due to its size. Merely check the code here, and then skip to where we load _zip_by_naics_ from csv.

Load the enormous file:

In [ ]:
zip_buss = pd.read_table(os.path.join('~\\Dropbox\\Documents\\Misc', 'CB1500CZ21.dat'), sep="|")

Keep only ZIP codes that are in the SBA data.

In [200]:
zip_clean = zip_buss[zip_buss['ZIPCODE'].isin(clean['borrzip'].unique())]

Check file length after trimming.

In [201]:
len(zip_clean)

2827380

The definition of 'small business' is '500 or fewer employees.' Let's filter our data accordingly.  Note that this file still has all the NAICS code breakdowns, which I keep because it might be useful for a future analysis.

In [ ]:
emp_sizes = ['Establishments with 1 to 4 employees',
             'Establishments with 5 to 9 employees',
             'Establishments with 10 to 19 employees',
             'Establishments with 20 to 49 employees',
             'Establishments with 50 to 99 employees',
             'Establishments with 100 to 249 employees',
             'Establishments with 250 to 499 employees']

These are the columns we care about:

In [ ]:
cols = ['ZIPCODE',
        'GEO_ID',
        'NAICS2012',
        'NAICS2012_TTL',
        'ESTAB']

Create the filtered file.

In [202]:
zip_size = pd.DataFrame(zip_clean[zip_clean['EMPSZES_TTL'].isin(emp_sizes)][cols].groupby(['ZIPCODE',
                                                                                          'GEO_ID',
                                                                                          'NAICS2012',
                                                                                          'NAICS2012_TTL']).ESTAB.sum())

Send filtered file to CSV.

In [203]:
zip_size.to_csv(os.path.join('Data', 'zip_by_naics.csv'))

Load file from CSV.

In [59]:
zip_by_naics = pd.read_csv(os.path.join('Data', 'zip_by_naics.csv'))

### IRS tax return by ZIP code data from 2014  

Income and tax data obtained from: https://www.irs.gov/uac/soi-tax-stats-individual-income-tax-statistics-2014-zip-code-data-soi  
Data dictionary is there too, but here is the downlooad link: https://www.irs.gov/pub/irs-soi/14zpdoc.doc

In [60]:
zip_irs = pd.read_csv(os.path.join('Data', '14_irs_zip.csv'))

There is A LOT of data, but for now, we'll only look at the mean adjusted grosss income by ZIP code.  
Note: `mean_agi` is calculated as _adjusted gross income_ divided by the _total number of returns_ plus the _number of joint returns_ (in order to obtain the mean AGI per taxable person).

In [61]:
zip_irs['mean_agi'] = (zip_irs['A00100']/(zip_irs['N1']+zip_irs['MARS2'])*1000).map(lambda x: int(x))

In [62]:
zip_agi = zip_irs[['mean_agi', 'ZIPCODE']]

### Combine SBA, total businesses, and IRS data into aggregated file

Merge SBA data for total SBA loans, 540 loans, and 7a loans.

In [63]:
agg = pd.concat([clean_tots, clean_tots_540, clean_tots_7a], axis = 1)
agg.rename(columns={0:'total_SBA', 1:'540', 2:'7a'},inplace=True)

Merge SBA data with NAICS data. We do a left join to keep all SBA data, and we only take the aggregated NAICS data.

In [64]:
agg2 = pd.merge(agg, zip_by_naics[zip_by_naics['NAICS2012']=='0'][['ESTAB', 'GEO_ID', 'ZIPCODE']], 
         left_index=True, right_on='ZIPCODE', how='left')

In [65]:
agg2 = agg2.set_index('ZIPCODE')

Merge data with IRS data. We again do a left join to keep all SBA data.

In [66]:
agg3 = pd.merge(agg2, zip_agi, left_index=True, right_on='ZIPCODE', how='left')

Zip IRS data has 51 records for zip code 0 and 99999, and plus these are useless zip codes, so we remove them.

In [67]:
agg3 = agg3[(agg3['ZIPCODE'] <> 99999) & (agg3['ZIPCODE'] <> 0)]

Get preliminary metrics, which are the ratios of # of SBA loans (total, 540, and 7a) to total number of businesses.

In [68]:
agg3['SBA_per_Small'] = agg3['total_SBA']/agg3['ESTAB']

In [69]:
agg3['540_per_Small'] = agg3['540']/agg3['ESTAB']

In [70]:
agg3['7a_per_Small'] = agg3['7a']/agg3['ESTAB']

For easier exploratory analysis, we will bucket all the key metrics into 8 distinct quantiles.

In [74]:
# Ratio of all SBA loans to number of small businesses
agg3 = agg3.assign(tot_ratio_buckets = pd.qcut(agg3['SBA_per_Small'], 8)) # original buckets
agg3 = agg3.assign(tot_ratio_buckets_rank = pd.qcut(agg3['SBA_per_Small'], 8, labels = False)) # buckets labeled by ranking
agg3['tot_ratio_buckets_rank'] = agg3['tot_ratio_buckets_rank'] + 1 # we want them written as 1-8, not 0-7
# Ratio of 540 loans to number of small businesses
agg3 = agg3.assign(_540_ratio_buckets = pd.qcut(agg3['540_per_Small'], 8)) # original buckets
agg3 = agg3.assign(_540_ratio_buckets_rank = pd.qcut(agg3['540_per_Small'], 8, labels = False)) # buckets labeled by ranking
agg3['_540_ratio_buckets_rank'] = agg3['_540_ratio_buckets_rank'] + 1# we want them written as 1-8, not 0-7
# Ratio of 7a loans to number of small businesses
agg3 = agg3.assign(_7a_ratio_buckets = pd.qcut(agg3['7a_per_Small'], 8)) # original buckets
agg3 = agg3.assign(_7a_ratio_buckets_rank = pd.qcut(agg3['7a_per_Small'], 8, labels = False)) # buckets labeled by ranking
agg3['_7a_ratio_buckets_rank'] = agg3['_7a_ratio_buckets_rank'] + 1# we want them written as 1-8, not 0-7
# Mean AGI buckets
agg3 = agg3.assign(mean_agi_buckets = pd.qcut(agg3['mean_agi'], 8)) # original buckets
agg3 = agg3.assign(mean_agi_buckets_rank = pd.qcut(agg3['mean_agi'], 8, labels = False)) # buckets labeled by ranking
agg3['mean_agi_buckets_rank'] = agg3['mean_agi_buckets_rank'] + 1# we want them written as 1-8, not 0-7

Send combined file to CSV.

In [77]:
agg3.to_csv(os.path.join('Data', 'combined.csv'), index=False)